# Обучение моделей с помощью Экспериментов Azure ML

__Цель лабораторной работы:__

- обучить модель машинного обучения, используя Эксперименты (Experiments) Azure ML
- зарегистрировать обученную модель.

## Подготовка среды

Импорт необходимых модулей и проверка версии Azure ML SDK:

In [4]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.core.datastore import Datastore
from azureml.train.sklearn import SKLearn

from azureml.widgets import RunDetails

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.14.0


Получим конфигурацию эксперимента: 

In [5]:
%run core.py

config = get_experiment_config('lab_4B')
init_experiment(config)
experiment_dir = get_experiment_dir(config)

config

Experiment train-model-experiment was initialized successfully.


{'experiment_name': 'train-model-experiment',
 'working_subdir': 'train-model-experiment-lab',
 'core': {'expriments_root_dir': 'expriments/',
  'datastore_name': 'aml_ws_datastore_v2',
  'dataset_name': 'diabetes-data',
  'ml_cluster_name': 'aml-ws-cluster',
  'ml_model_name': 'diabetes-predict-model'}}

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [6]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: aml-workshop.


## Подготовка данных для Эксперимента

### Загрузка данных

Воспользуемся данными, загружеными в Хранилище данных в [лабороторной 2B](02B-datastores-and-datasets.ipynb):

In [8]:
data_ds = ws.datasets.get(config['core']['dataset_name'])
print(f'Used dataset {data_ds.name}: {data_ds.description}')

Used dataset diabetes-data: Diabetes Disease Database


### Скрипт для обучения модели

Скопируем скрипт и содержимое директории Эксперимента:

In [15]:
!cp scripts/train_model.py $experiment_dir
!ls $experiment_dir

train_model.py


## Создадим и запустим Эксперимент

In [ ]:
# Create an experiment
experiment = Experiment(workspace=ws, name=config['experiment_name'])

# Create an SKLearn estimator
estimator = SKLearn(source_directory=experiment_dir,
                    inputs=[data_ds.as_named_input('data')], 
                    entry_script='train_model.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local',
                    pip_packages=['matplotlib']
                    )


# Run the experiment and wait result
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

See training details:

In [ ]:
RunDetails(run).show()

Просмотрим метрики и выходные данные:

In [ ]:
metrics = run.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

for file in run.get_file_names():
    print(file)

## Регистрируем обученную модель

Копируем путь до обученной модели (`outputs/model.pkl`), чтобы зарегистрировать ее. Обычно это последний файл:

In [14]:
model_path = run.get_file_names()[-1]
model_path

'outputs/model.pkl'

In [ ]:
run.register_model(model_path=model_path, model_name=config['core']['ml_model_name'],
                   tags={'Lab':'4B'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Просмотрим список зарегистрированных моделей:

In [17]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print(f'\t {tag_name}: {tag}')
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print(f'\t {prop_name}: {prop}')
        
    print('\n')

diabetes-predict-model v1
	 Lab: 4B
	 0.8576982541474856: AUC
	 0.7876666666666666: Accuracy




Проcмотрите зарегистрированную модель на портале Azure ML в разделе: `<your_workspace> > Models > diabetes_model`.